In [1]:
import geopandas as gpd

gdf = gpd.read_file("geojson/ACM_building.geojson") #coords in EPSG:4326 (long/lat)
gdf_proj = gdf.to_crs(epsg=3857) #convert geodataframe coord from long/lat to meters

In [61]:
ACM = gdf_proj.geometry 

ACM_edited = ACM.simplify_coverage(2.9)  # 100 meters tolerance


In [62]:
print(ACM_edited.crs)
ACM_edited_4326 = ACM_edited.to_crs("EPSG:4326")
print(ACM_edited_4326.crs)

EPSG:3857
EPSG:4326


In [63]:
def pt(v):
    print(type(v))
    
def count_coords(geom):
    if geom.geom_type == 'Polygon':
        return len(geom.exterior.coords)
    elif geom.geom_type == 'MultiPolygon':
        return sum(len(poly.exterior.coords) for poly in geom.geoms)
    else:
        return None

num_points_before = ACM.apply(count_coords)
num_points_after  = ACM_edited.apply(count_coords)

print("Before simplification:\n", num_points_before)
print("\nAfter simplification:\n", num_points_after)

Before simplification:
 0    105
Name: geometry, dtype: int64

After simplification:
 0    52
dtype: int64


In [64]:
gdf = gpd.GeoDataFrame(geometry=ACM_edited_4326)
gdf.to_file("output.geojson", driver="GeoJSON")

In [71]:
import geopandas as gpd
from shapely.geometry import Polygon, MultiPolygon, LinearRing

# ----- Step 1: Load your GeoJSON -----
gdf = gpd.read_file("geojson/ACM_building.geojson")

geom_series = gdf.geometry

# ----- Step 2: Chaikin smoothing function -----
def chaikin_smooth(coords, iterations=2):
    """
    coords: list of (x, y) tuples
    iterations: number of smoothing passes
    Returns new list of smoothed coordinates
    """
    for _ in range(iterations):
        new_coords = []
        for i in range(len(coords)-1):
            p0, p1 = coords[i], coords[i+1]
            Q = (0.75*p0[0] + 0.25*p1[0], 0.75*p0[1] + 0.25*p1[1])
            R = (0.25*p0[0] + 0.75*p1[0], 0.25*p0[1] + 0.75*p1[1])
            new_coords += [Q, R]
        # Close the ring
        new_coords.append(new_coords[0])
        coords = new_coords
    return coords

# ----- Step 3: Apply smoothing to each geometry -----
def smooth_polygon(geom, iterations=2, simplify_tol=None):
    if geom.geom_type == 'Polygon':
        # Simplify first (optional)
        if simplify_tol:
            geom = geom.simplify(simplify_tol, preserve_topology=True)
        # Apply Chaikin smoothing to exterior
        new_exterior = LinearRing(chaikin_smooth(list(geom.exterior.coords), iterations))
        # Smooth interiors (holes) if any
        new_interiors = [LinearRing(chaikin_smooth(list(ring.coords), iterations)) for ring in geom.interiors]
        return Polygon(new_exterior, new_interiors)
    elif geom.geom_type == 'MultiPolygon':
        return MultiPolygon([smooth_polygon(p, iterations, simplify_tol) for p in geom.geoms])
    else:
        return geom

# Example: 2 Chaikin iterations, optional 3-meter simplification
geom_series_proj = geom_series.to_crs(epsg=3857)
smoothed_series = geom_series_proj.apply(lambda g: smooth_polygon(g, iterations=1, simplify_tol=3))
smoothed_series_4326 = smoothed_series.to_crs(epsg=4326)
# ----- Step 4: Save as GeoJSON -----
gdf_smoothed = gpd.GeoDataFrame(geometry=smoothed_series_4326, crs=gdf.crs)
gdf_smoothed.to_file("smoothed_output.geojson", driver="GeoJSON")
